In [2]:
import math
%load_ext Cython

In [ ]:
%%cython -a
import numpy as np
cimport numpy as np
cimport cython
from libc.math cimport exp, sqrt, pow
np.import_array()
def binomial_model(float SO, float K, float T, float r, float sigma, int N, bytes option_type, bint american):
    cdef float dt = T / N # time intervals
    cdef float u = exp(sigma * sqrt(dt)) # up factor
    cdef float d = 1.0 / u
    cdef float p = (exp(r * dt)-d) / (u - d) # risk neutral probability
    
    

    cdef np.ndarray[np.float64_t, ndim = 2] prices = np.zeros((N + 1, N + 1))
    cdef np.ndarray[np.float64_t, ndim = 2] option_values = np.zeros((N + 1, N + 1))

    cdef int i, j, t

    prices[0, 0] = SO

    for t in range(N + 1):
        for i in range(t + 1):        
            prices[i, t] = SO * pow(u, (t - i)) * pow(d, i) # calucualtes the finincal options value at maturity

    for i in range(N + 1):
        if option_type == b"c":
            option_values[i,N] = max(prices[i, N] - K, 0) # if a call option take the masimimum of the option value
        elif option_type == b"p":
            option_values[i, N] = max(K - prices[i, N], 0)

    for j in range(N - 1, -1, -1):
        for i in range(j + 1):
            continuation_value = exp(-r * dt) * (p * option_values[i, j + 1] + (1 - p) * option_values[i + 1, j+1]) # backward induction to calculate the option value at each node
            if american:
                if option_type == b"c":
                    exercise_value = max(prices[i, j] - K, 0) # if a call option take the maximum of the option value
                elif option_type == b"p":
                    exercise_value = max(K - prices[i, j], 0)

                option_values[i, j] = max(exercise_value, continuation_value)
            else:
                option_values[i, j] = continuation_value
    return option_values[0, 0]
    





Content of stderr:
In file included from /home/justin/quant_project/.venv/lib/python3.12/site-packages/numpy/_core/include/numpy/ndarraytypes.h:1913,
                 from /home/justin/quant_project/.venv/lib/python3.12/site-packages/numpy/_core/include/numpy/ndarrayobject.h:12,
                 from /home/justin/quant_project/.venv/lib/python3.12/site-packages/numpy/_core/include/numpy/arrayobject.h:5,
                 from /home/justin/.cache/ipython/cython/_cython_magic_c6df76937683745e7fbaa22f9aab29a3245084691d73bc09c2c5351a8ab4bd06.c:1152:
/home/justin/quant_project/.venv/lib/python3.12/site-packages/numpy/_core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~

In [37]:
binomial_model(100.0, 100.0, 1.0, 0.05, 0.2, 100, b"c", True)

10.430611336981784